# K-Drama Browser - a tool for getting quick summaries of K-Drama's based on user sentiment and reviews

Jeff Barney, MSAI-495: Generative AI Text Project

In [2]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
import re
import string
from IPython.display import display, HTML

import tensorflow as tf
from tensorflow.keras import layers, models, losses, callbacks

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


2025-06-02 20:57:58.260952: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-02 20:58:01.778991: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-06-02 20:58:01.779038: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-06-02 20:58:01.888290: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-06-02 20:58:02.019324: I tensorflow/core/platform/cpu_feature_guar

## 0. Parameters

In [3]:
VOCAB_SIZE = 100000
MAX_LEN = 80
EMBEDDING_DIM = 256
KEY_DIM = 256
N_HEADS = 2
FEED_FORWARD_DIM = 256
VALIDATION_SPLIT = 0.2
SEED = 42
LOAD_MODEL = False
BATCH_SIZE = 32
EPOCHS = 5

## 1. Load the data

In [5]:
import kagglehub

raw_data_path = kagglehub.dataset_download("chanoncharuchinda/korean-drama-2015-23-actor-and-reviewmydramalist") + "/reviews.csv"

/home/nee0873/.conda/envs/text-project/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


100%|██████████| 8.40M/8.40M [00:00<00:00, 57.1MB/s]

Extracting files...


In [6]:
# Filter the dataset, remove colons
raw_data_df = pd.read_csv(raw_data_path)
cols_to_drop = [col for col in raw_data_df.columns if col not in ['title', 'review_text']]
filtered_data = raw_data_df.drop(columns=cols_to_drop)
filtered_data['review_text'] = filtered_data['review_text'].str.replace(':', '-')

In [7]:
# Combine the review and titles
combined_data = filtered_data.apply(lambda row: 'K-Drama title : ' + ' : K-Drama review : '.join(row.astype(str)), axis=1).tolist()
n_reviews = len(combined_data)
print(f"{n_reviews} reviews loaded")

10625 reviews loaded


In [8]:
combined_data[25]

"K-Drama title : Star Struck : K-Drama review : More like a glimmer rather than being starstruck  I've said it before and I'll say it again- This should have been a movie. Or, at least, been released all at once. The cut-off of many episodes felt unsatisfying and, overall, just got in the way of the flow of the story.Also, it seemed like scenes were missing, and I'm sure a longer runtime would have added more depth to it . . . However--strangely--I'm not mad about it. Maybe it's the indie vibes and solid acting that won me over and made me see past its flaws.Just about Zuho- I do have to bring up how great it was to have him, being the fairly famous K-Pop Idol he is, actually speak out against the hate he was getting for acting in a BL, and not just that, but openly voice his support, love, and respect for the LGBT+ community in his response."

## 2. Tokenize the data

In [9]:
# Pad the punctuation, to treat them as separate 'words'
def pad_punctuation(s):
    s = re.sub(f"([{string.punctuation}, '\n'])", r" \1 ", s)
    s = re.sub(" +", " ", s)
    return s


text_data = [pad_punctuation(x) for x in combined_data]

In [10]:
# Display an example of a review
text_data[25]

"K - Drama title : Star Struck : K - Drama review : More like a glimmer rather than being starstruck I ' ve said it before and I ' ll say it again - This should have been a movie . Or , at least , been released all at once . The cut - off of many episodes felt unsatisfying and , overall , just got in the way of the flow of the story . Also , it seemed like scenes were missing , and I ' m sure a longer runtime would have added more depth to it . . . However - - strangely - - I ' m not mad about it . Maybe it ' s the indie vibes and solid acting that won me over and made me see past its flaws . Just about Zuho - I do have to bring up how great it was to have him , being the fairly famous K - Pop Idol he is , actually speak out against the hate he was getting for acting in a BL , and not just that , but openly voice his support , love , and respect for the LGBT + community in his response . "

In [11]:
# Convert to a Tensorflow Dataset
text_ds = (
    tf.data.Dataset.from_tensor_slices(text_data)
    .batch(BATCH_SIZE)
    .shuffle(1000)
)

2025-06-02 21:06:16.001188: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2348] TensorFlow was not built with CUDA kernel binaries compatible with compute capability 9.0. CUDA kernels will be jit-compiled from PTX, which could take 30 minutes or longer.
2025-06-02 21:06:16.007894: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2348] TensorFlow was not built with CUDA kernel binaries compatible with compute capability 9.0. CUDA kernels will be jit-compiled from PTX, which could take 30 minutes or longer.
2025-06-02 21:06:16.165881: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 78763 MB memory:  -> device: 0, name: NVIDIA H100 80GB HBM3, pci bus id: 0000:d4:00.0, compute capability: 9.0


In [12]:
# Create a vectorisation layer
vectorize_layer = layers.TextVectorization(
    standardize="lower",
    max_tokens=VOCAB_SIZE,
    output_mode="int",
    output_sequence_length=MAX_LEN + 1,
)

In [13]:
# Adapt the layer to the training set
vectorize_layer.adapt(text_ds)
vocab = vectorize_layer.get_vocabulary()

In [14]:
# Display some token:word mappings
for i, word in enumerate(vocab[:10]):
    print(f"{i}: {word}")

0: 
1: [UNK]
2: .
3: the
4: ,
5: and
6: i
7: a
8: to
9: of


In [15]:
# Display the same example converted to ints
example_tokenised = vectorize_layer(text_data[25])
print(example_tokenised.numpy())

2025-06-02 21:09:31.614948: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


[   27    10    13    63    20   776  3793    20    27    10    13    54
    20    55    34     7 11831   321   122   127 30869     6    11   169
   351    12   261     5     6    11   277   146    12   172    10    18
   204    33   124     7   805     2    66     4    51   322     4   124
  1891    39    51   353     2     3   972    10   197     9   126    96
   140  3872     5     4   163     4    44   173    15     3   108     9
     3  1555     9     3    40     2    82     4    12]


## 3. Create the Training Set

In [16]:
# Create the training set of reviews and the same text shifted by one word
def prepare_inputs(text):
    text = tf.expand_dims(text, -1)
    tokenized_sentences = vectorize_layer(text)
    x = tokenized_sentences[:, :-1]
    y = tokenized_sentences[:, 1:]
    return x, y


train_ds = text_ds.map(prepare_inputs)

In [17]:
example_input_output = train_ds.take(1).get_single_element()

In [18]:
# Example Input
example_input_output[0][0]

<tf.Tensor: shape=(80,), dtype=int64, numpy=
array([  27,   10,   13,   63,   20, 2092,  814,   20,   27,   10,   13,
         54,   20,   59,   18,   14,  144,   92,  773,  348,    4,  134,
         49,   36,  341,    7,  136,    9,  154,    8,   34,   61,    3,
       2092,  814,    4,  745,   21,   42,   18,   14,   28,   57,   92,
       1865,   29,    6,   16,  552,  834,    5,  514,    2,   12,   11,
         23,   28,   17,   18,   13,   14,  153,    4,   12,   11,   23,
         44,   17,   12,   11,   23,   81, 1585, 1052,    8,  126,  123,
         22,  697,  839])>

In [19]:
# Example Output (shifted by one token)
example_input_output[1][0]

<tf.Tensor: shape=(80,), dtype=int64, numpy=
array([  10,   13,   63,   20, 2092,  814,   20,   27,   10,   13,   54,
         20,   59,   18,   14,  144,   92,  773,  348,    4,  134,   49,
         36,  341,    7,  136,    9,  154,    8,   34,   61,    3, 2092,
        814,    4,  745,   21,   42,   18,   14,   28,   57,   92, 1865,
         29,    6,   16,  552,  834,    5,  514,    2,   12,   11,   23,
         28,   17,   18,   13,   14,  153,    4,   12,   11,   23,   44,
         17,   12,   11,   23,   81, 1585, 1052,    8,  126,  123,   22,
        697,  839,    2])>

## 4. Create the causal attention mask function

In [20]:
def causal_attention_mask(batch_size, n_dest, n_src, dtype):
    i = tf.range(n_dest)[:, None]
    j = tf.range(n_src)
    m = i >= j - n_src + n_dest
    mask = tf.cast(m, dtype)
    mask = tf.reshape(mask, [1, n_dest, n_src])
    mult = tf.concat(
        [tf.expand_dims(batch_size, -1), tf.constant([1, 1], dtype=tf.int32)], 0
    )
    return tf.tile(mask, mult)


np.transpose(causal_attention_mask(1, 10, 10, dtype=tf.int32)[0])

array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [0, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [0, 0, 1, 1, 1, 1, 1, 1, 1, 1],
       [0, 0, 0, 1, 1, 1, 1, 1, 1, 1],
       [0, 0, 0, 0, 1, 1, 1, 1, 1, 1],
       [0, 0, 0, 0, 0, 1, 1, 1, 1, 1],
       [0, 0, 0, 0, 0, 0, 1, 1, 1, 1],
       [0, 0, 0, 0, 0, 0, 0, 1, 1, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1]], dtype=int32)

## 5. Create a Transformer Block layer

In [21]:
class TransformerBlock(layers.Layer):
    def __init__(self, num_heads, key_dim, embed_dim, ff_dim, dropout_rate=0.1):
        super(TransformerBlock, self).__init__()
        self.num_heads = num_heads
        self.key_dim = key_dim
        self.embed_dim = embed_dim
        self.ff_dim = ff_dim
        self.dropout_rate = dropout_rate
        self.attn = layers.MultiHeadAttention(
            num_heads, key_dim, output_shape=embed_dim
        )
        self.dropout_1 = layers.Dropout(self.dropout_rate)
        self.ln_1 = layers.LayerNormalization(epsilon=1e-6)
        self.ffn_1 = layers.Dense(self.ff_dim, activation="relu")
        self.ffn_2 = layers.Dense(self.embed_dim)
        self.dropout_2 = layers.Dropout(self.dropout_rate)
        self.ln_2 = layers.LayerNormalization(epsilon=1e-6)

    def call(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size = input_shape[0]
        seq_len = input_shape[1]
        causal_mask = causal_attention_mask(
            batch_size, seq_len, seq_len, tf.bool
        )
        attention_output, attention_scores = self.attn(
            inputs,
            inputs,
            attention_mask=causal_mask,
            return_attention_scores=True,
        )
        attention_output = self.dropout_1(attention_output)
        out1 = self.ln_1(inputs + attention_output)
        ffn_1 = self.ffn_1(out1)
        ffn_2 = self.ffn_2(ffn_1)
        ffn_output = self.dropout_2(ffn_2)
        return (self.ln_2(out1 + ffn_output), attention_scores)

    def get_config(self):
        config = super().get_config()
        config.update(
            {
                "key_dim": self.key_dim,
                "embed_dim": self.embed_dim,
                "num_heads": self.num_heads,
                "ff_dim": self.ff_dim,
                "dropout_rate": self.dropout_rate,
            }
        )
        return config

## 6. Create the Token and Position Embedding

In [22]:
class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, max_len, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.max_len = max_len
        self.vocab_size = vocab_size
        self.embed_dim = embed_dim
        self.token_emb = layers.Embedding(
            input_dim=vocab_size, output_dim=embed_dim
        )
        self.pos_emb = layers.Embedding(input_dim=max_len, output_dim=embed_dim)

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions

    def get_config(self):
        config = super().get_config()
        config.update(
            {
                "max_len": self.max_len,
                "vocab_size": self.vocab_size,
                "embed_dim": self.embed_dim,
            }
        )
        return config

## 7. Build the Transformer model

In [23]:
inputs = layers.Input(shape=(None,), dtype=tf.int32)
x = TokenAndPositionEmbedding(MAX_LEN, VOCAB_SIZE, EMBEDDING_DIM)(inputs)
x, attention_scores = TransformerBlock(
    N_HEADS, KEY_DIM, EMBEDDING_DIM, FEED_FORWARD_DIM
)(x)
outputs = layers.Dense(VOCAB_SIZE, activation="softmax")(x)
gpt = models.Model(inputs=inputs, outputs=[outputs, attention_scores])
gpt.compile("adam", loss=[losses.SparseCategoricalCrossentropy(), None])

In [24]:
gpt.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None)]            0         
                                                                 
 token_and_position_embeddi  (None, None, 256)         25620480  
 ng (TokenAndPositionEmbedd                                      
 ing)                                                            
                                                                 
 transformer_block (Transfo  ((None, None, 256),       658688    
 rmerBlock)                   (None, 2, None, None))             
                                                                 
 dense_2 (Dense)             (None, None, 100000)      25700000  
                                                                 
Total params: 51979168 (198.28 MB)
Trainable params: 51979168 (198.28 MB)
Non-trainable params: 0 (0.00 Byte)
_________________

## 8. Train the Transformer

In [25]:
# Create a TextGenerator checkpoint
class TextGenerator(callbacks.Callback):
    def __init__(self, index_to_word, top_k=10):
        self.index_to_word = index_to_word
        self.word_to_index = {
            word: index for index, word in enumerate(index_to_word)
        }

    def sample_from(self, probs, temperature):
        probs = probs ** (1 / temperature)
        probs = probs / np.sum(probs)
        return np.random.choice(len(probs), p=probs), probs

    def generate(self, start_prompt, max_tokens, temperature):
        start_tokens = [
            self.word_to_index.get(x, 1) for x in start_prompt.split()
        ]
        sample_token = None
        info = []
        while len(start_tokens) < max_tokens and sample_token != 0:
            x = np.array([start_tokens])
            y, att = self.model.predict(x, verbose=0)
            sample_token, probs = self.sample_from(y[0][-1], temperature)
            info.append(
                {
                    "prompt": start_prompt,
                    "word_probs": probs,
                    "atts": att[0, :, -1, :],
                }
            )
            start_tokens.append(sample_token)
            start_prompt = start_prompt + " " + self.index_to_word[sample_token]
        print(f"\ngenerated text:\n{start_prompt}\n")
        return info

    def on_epoch_end(self, epoch, logs=None):
        self.generate("K-Drama title", max_tokens=80, temperature=1.0)

In [26]:
# Tokenize starting prompt
text_generator = TextGenerator(vocab)

In [28]:
gpt.fit(
    train_ds,
    epochs=EPOCHS,
    callbacks=[text_generator],
)

Epoch 1/5
332/333 [============================>.] - ETA: 0s - loss: 3.2972 - dense_2_loss: 3.2972
generated text:
K-Drama title : k - pop me screaming and mister : k - drama review : i watch this show , with a mess , i know that i did a revenge story , i . the words war of why the laughter in spoiling the entire quality of this drama up it off as it has see how each other and kim min ho was . the music was amazing and jin sung kyung po returns after he happened

333/333 [==============================] - 9s 27ms/step - loss: 3.2977 - dense_2_loss: 3.2977
Epoch 2/5
332/333 [============================>.] - ETA: 0s - loss: 3.1066 - dense_2_loss: 3.1066
generated text:
K-Drama title : k - drama review : left me this week : i was a little heartfelt drama that it was like it came up ending . . it was all kind of the topic realised had something interesting . obsession with cringe , things were honestly , but they had much of craziness that didn ' t seem to be fooled by the ' s not the bes

# 9. Generate text using the Transformer

In [29]:
def print_probs(info, vocab, top_k=5):
    for i in info:
        highlighted_text = []
        for word, att_score in zip(
            i["prompt"].split(), np.mean(i["atts"], axis=0)
        ):
            highlighted_text.append(
                '<span style="background-color:rgba(135,206,250,'
                + str(att_score / max(np.mean(i["atts"], axis=0)))
                + ');">'
                + word
                + "</span>"
            )
        highlighted_text = " ".join(highlighted_text)
        display(HTML(highlighted_text))

        word_probs = i["word_probs"]
        p_sorted = np.sort(word_probs)[::-1][:top_k]
        i_sorted = np.argsort(word_probs)[::-1][:top_k]
        for p, i in zip(p_sorted, i_sorted):
            print(f"{vocab[i]}:   \t{np.round(100*p,2)}%")
        print("--------\n")

In [30]:
info = text_generator.generate(
    "K-Drama title : Unlock My Boss : K-Drama review : ", max_tokens=80, temperature=1.0
)


generated text:
K-Drama title : Unlock My Boss : K-Drama review :  ruin a thousand enough time : k - drama review : the concept is cute and predictable . after few episodes i think the show is simple and i can confirm that it had good concept without taking the whole vibe and why you are not regret watching it . this is a pretty good feel the sense . if your thing the performances is that of mmh is one



In [31]:
info = text_generator.generate(
    "K-Drama title : Blueming : K-Drama review : ", max_tokens=80, temperature=0.5
)


generated text:
K-Drama title : Blueming : K-Drama review :  the paradise in a drama review : a lot of people who are looking for a very good drama . i was looking forward to this drama . i was hooked . i initially thought i was a huge fan of seo yeon and ji hoon and the characters were amazing . i was looking forward to this drama . i loved how they were in high school and the cast was



In [32]:
info = text_generator.generate(
    "K-Drama title : Star Struck : K-Drama review : ", max_tokens=80, temperature=0.5
)
print_probs(info, vocab)


generated text:
K-Drama title : Star Struck : K-Drama review :  paradise in a drama review : not good drama . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . calmly one . .



k:   	84.27%
paradise:   	4.19%
nassna:   	3.65%
beneath:   	2.75%
the:   	1.48%
--------



in:   	51.5%
inni:   	21.1%
-:   	3.82%
looking:   	3.74%
::   	2.45%
--------



a:   	70.68%
the:   	10.54%
time:   	9.91%
love:   	6.97%
-:   	1.02%
--------



drama:   	58.93%
mere:   	15.16%
terminally:   	5.27%
thousand:   	4.49%
::   	4.36%
--------



...

.:   	99.93%
,:   	0.03%
and:   	0.01%
~:   	0.01%
?:   	0.01%
--------



.:   	99.88%
,:   	0.03%
~:   	0.02%
okiee:   	0.02%
calmly:   	0.01%
--------



":   	26.69%
theories:   	8.74%
wanting:   	7.71%
gifted:   	6.97%
pride:   	5.81%
--------



.:   	43.34%
of:   	40.0%
-:   	4.06%
?:   	3.45%
):   	2.22%
--------



.:   	97.04%
i:   	1.0%
lol:   	0.79%
?:   	0.54%
):   	0.18%
--------

